In [ ]:
from bs4 import BeautifulSoup
# from lxml import etree

#read tags from xml file
with open('./plant_labels.xml', 'r') as f:
    label_tree = f.read()
 
# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object
soup = BeautifulSoup(label_tree, "xml")
#xml tree structure <Data> -> <Image> -> filename,ids, etc
# data_element = etree.Element("Data")


### DataParser

Read plant label xml file, parse data to be added to db and save to a dict.

In [ ]:
import pandas as pd
import requests

#this function reads an image tag and makes it's data a dictionary. It returns a dictionary   
def build_row_dict(tree_data):

    #build image_attributes_dict
    #dict containing species, family, classId, etc. To be nested inside row_data_dict, paired with filenames
    #Keys = Species, Content, Genus, Family, ObservationId, ClassId
    #Values = associated key:val pair
    image_attributes_list = ["PlantId","Species", "Content", "Genus","Family", "ObservationId","ClassId" ]
    image_attributes_values = [
        tree_data.FileName.get_text(),
        tree_data.Species.get_text(),
        tree_data.Content.get_text(),
        tree_data.Genus.get_text(),
        tree_data.Family.get_text(),
        tree_data.ObservationId.get_text(),
        tree_data.ClassId.get_text()
    ]
    row_data_dict = { key:value for (key,value) in zip(image_attributes_list, image_attributes_values)}
    return row_data_dict

In [ ]:
#navigate through xml tree, tranverse down from "Image" tag to "Imange tag"

plant_label_data_array = []
rows= soup.find_all("Image")

# rows.find("Image")
for image_tag_data in rows:
    plant_label_data_array.append(build_row_dict(image_tag_data))


In [ ]:
plant_label_data_array

### DataExporter

Exports data to plantdb

In [ ]:
from os import getenv
from dotenv import load_dotenv

load_dotenv()

config = {
    "DB_CONNECTION_STRING": getenv("DB_CONNECTION_STRING")
}

In [ ]:
import pymongo
import certifi
#mongodb library

myclient = pymongo.MongoClient(config["DB_CONNECTION_STRING"], tlsCAFile=certifi.where())
mydb = myclient["plantdb"]
mycol = mydb["plant_labels"]

mylist = plant_label_data_array

x = mycol.insert_many(mylist)
#print list of the _id values of the inserted documents:
# print(x.inserted_ids)